In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test/255.0

x_train = x_train[..., tf.newaxis]#.astype(int)
x_test = x_test[..., tf.newaxis]

In [3]:
x_train[0].shape

(28, 28, 1)

In [4]:
x_train[0][0,0,0]

0.0

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [15]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation="relu")
    self.flatten = Flatten()
    self.d1 = Dense(128, activation="relu")
    self.d2 = Dense(10, activation="softmax")

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d2(x)
    return x

model = MyModel()



In [16]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [17]:
train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [19]:
@tf.function
def train(images, labels):
    with tf.GradientTape() as tape:
      pred = model(images)
      loss = loss_function(labels, pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, pred)

In [20]:
@tf.function
def test(images, labels):
    pred = model(images)
    loss = loss_function(labels, pred)
    test_loss(loss)
    test_accuracy(labels, pred)


In [21]:
epochs = 5

for epoch in range(epochs):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_dataset:
    train(images, labels)

    #test
  for image, label in test_dataset:
    test(image, label)

  print("epoch:", epoch+1,
        f"Train Loss: {train_loss.result()}",
        f"Train accuracy: {train_accuracy.result()}",
        f"Test Loss: {test_loss.result()}",
        f"Test accuracy, {test_accuracy.result()}")

C:\Users\Alireza\AppData\Roaming\Python\Python311\site-packages\keras\backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


epoch: 1 Train Loss: 0.13404367864131927 Train accuracy: 0.9606500267982483 Test Loss: 0.06659740209579468 Test accuracy, 0.9750999808311462
epoch: 2 Train Loss: 0.04259447380900383 Train accuracy: 0.986549973487854 Test Loss: 0.056854795664548874 Test accuracy, 0.9805999994277954
epoch: 3 Train Loss: 0.02194853313267231 Train accuracy: 0.9928500056266785 Test Loss: 0.056964028626680374 Test accuracy, 0.982699990272522
epoch: 4 Train Loss: 0.013190179131925106 Train accuracy: 0.9957833290100098 Test Loss: 0.07161098718643188 Test accuracy, 0.9805999994277954
epoch: 5 Train Loss: 0.008774977177381516 Train accuracy: 0.9969333410263062 Test Loss: 0.075578473508358 Test accuracy, 0.9793000221252441


In [12]:
import cv2
import numpy as np

image = cv2.imread("download.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = cv2.resize(image, (28, 28))
image = image[..., tf.newaxis]
image = image[tf.newaxis, ...]
image = image.astype("float32")

pred = model(image)
result = np.argmax(pred)
result

5

In [13]:
model.save("my_model")

INFO:tensorflow:Assets written to: my_model\assets


INFO:tensorflow:Assets written to: my_model\assets


In [14]:
model = tf.keras.models.load_model("my_model")